In [6]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [7]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [8]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [9]:
import gzip
import simplejson as json

In [10]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)
    
print(type(scripts))

<class 'list'>


This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [11]:
scripts[:3]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500},
 {'bnf_code': '0101021B0AAALAL',
  'items': 12,
  'practice': 'N81013',
  'bnf_name': 'Sod Algin/Pot Bicarb_Susp S/F',
  'nic': 64.51,
  'act_cost': 59.95,
  'quantity': 6300}]

In [12]:
x=5
y = x/2 if x%2 == 0 else x//2 + 1
print(y)

sorted_scripts = [script['items'] for script in scripts]
sorted_scripts.sort()
q = sorted_scripts[50/4 if 50%4 == 0 else 50//4 + 1]
print(q)

3
1


A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [13]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In [14]:
count = 0
for i in scripts:
    print(i["items"])
    count += 1
    if count == 3:
        break
print(sum([1,7]))
print(sum([script['items'] for script in scripts]))
import math
print(math.sqrt(sum([(script['items'] - 20)**2 for script in scripts])/len(scripts)))

2
1
12
8
4410054
34.18010264183149


In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [15]:
import math
def describe(key):
    
    length = len(scripts)
    total = sum([script[key] for script in scripts])
    avg = total/len(scripts)
    s = math.sqrt(sum([(script[key] - avg)**2 for script in scripts])/(length - 1))
    sorted_scripts = [script[key] for script in scripts]
    sorted_scripts.sort()
    q25 = sorted_scripts[(length/4) -1 if length%4 == 0 else (length//4) + 1 -1]
    med = sorted_scripts[(length//2) - 1] if length%2 == 0 else (sorted_scripts[(length//2) -1] + sorted_scripts[(length//2) + 1 -1])/2
    q75 = sorted_scripts[(length*3/4) -1] if length%4 == 0 else (sorted_scripts[(length*3//4)-1] + sorted_scripts[length*3//4 + 1 - 1])/2

    return (total, avg, s, q25, med, q75)

In [16]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]
print(summary)

[('items', (4410054, 11.522744731217633, 33.11220959820685, 1, 3, 8.0)), ('quantity', (316356836, 826.5883059943667, 3872.186073305146, 30, 120, 466.0)), ('nic', (29048309.790000338, 75.89844899484315, 197.57308474088356, 7.7, 22.62, 65.94)), ('act_cost', (27053937.599999707, 70.68748295124895, 183.26755837716715, 7.25, 21.24, 61.53))]


In [17]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [18]:
bnf_names = []
for script in scripts:
    if script['bnf_name'] not in bnf_names:
        bnf_names.append(script['bnf_name'])
assert(len(bnf_names) == 5619)

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [19]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script)

In [20]:
for key, value in groups.items():
    print(key, value)
    break
print(len(groups['Co-Magaldrox_Susp 195mg/220mg/5ml S/F']))

Co-Magaldrox_Susp 195mg/220mg/5ml S/F [{'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'N81013', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000}, {'bnf_code': '0101010G0AAABAB', 'items': 1, 'practice': 'N81029', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 2.99, 'act_cost': 2.78, 'quantity': 500}, {'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'N81088', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000}, {'bnf_code': '0101010G0AAABAB', 'items': 6, 'practice': 'A81017', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 20.93, 'act_cost': 19.45, 'quantity': 3500}, {'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'A81034', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000}, {'bnf_code': '0101010G0AAABAB', 'items': 10, 'practice': 'P85003', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [21]:
max_item = [("", 0)]
for key, value in groups.items():
    if len(value) >= max_item[0][1]:
        max_item = [(key, len(value))]

In [22]:
max_item = [("", 0)]
for key, value in groups.items():
    count = 0
    for item in value:
        count += item['items']
    if count > max_item[0][1]:
        max_item = [(key, count)]

In [23]:
for key, value in groups.items():
    print(type(len(value))) 
    print((max_item[0][1]))
    print((max_item[0][0]))
    print(max_item)
    break

<class 'int'>
113826
Omeprazole_Cap E/C 20mg
[('Omeprazole_Cap E/C 20mg', 113826)]


**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [24]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [25]:
def group_by_field(data, fields):
    groups = {}
    return groups

In [26]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = ...

assert test_max_item == max_item

AssertionError: 

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices'`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [27]:
practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        if practice['post_code'] < practice_postal[practice['code']]:
            practice_postal[practice['code']] = practice['post_code']
        else:
            pass
    else:
        practice_postal[practice['code']] = practice['post_code']
print(len(practice_postal))

10843


In [28]:
print(len(scripts))
print(len(practices))
lst = []
lst2 = []
rep = []
for prac in practices:
    if prac['code'] not in lst:
        lst.append(prac['code'])
    else:
        rep.append(prac['code'])
for script in scripts:
    if script['practice'] not in lst2:
        lst2.append(script['practice'])
print(len(lst))
print(len(lst2))
rep[0]

382726
12020
10843
494


'A81001'

In [29]:
for prac in practices:
    if prac['code'] == 'A81001':
        print(prac)

{'code': 'A81001', 'name': 'THE DENSHAM SURGERY', 'addr_1': 'THE HEALTH CENTRE', 'addr_2': 'LAWSON STREET', 'borough': 'STOCKTON ON TEES', 'village': 'CLEVELAND', 'post_code': 'TS18 1HU'}
{'code': 'A81001', 'name': 'THE DENSHAM SURGERY', 'addr_1': 'THE HEALTH CENTRE', 'addr_2': 'LAWSON STREET', 'borough': 'STOCKTON-ON-TEES', 'village': 'CLEVELAND', 'post_code': 'TS18 1HU'}


In [30]:
count_str = 0
count_list = 0
for practice in practices:
    if type(practice['post_code']) == str:
        count_str += 1
    if type(practice['post_code']) != str:
        count_list += 1
        
print(count_str)
print(count_list)

12020
0


**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [31]:
assert practice_postal['K82019'] == 'HP21 8TR'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [ ]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [32]:
joined = scripts[:]
for script in joined:
    script['post_code'] = practice_postal[script['practice']]
    
print(joined[:2])
print(len(joined))

[{'bnf_code': '0101010G0AAABAB', 'items': 2, 'practice': 'N81013', 'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F', 'nic': 5.98, 'act_cost': 5.56, 'quantity': 1000, 'post_code': 'SK11 6JL'}, {'bnf_code': '0101021B0AAAHAH', 'items': 1, 'practice': 'N81013', 'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F', 'nic': 1.95, 'act_cost': 1.82, 'quantity': 500, 'post_code': 'SK11 6JL'}]
382726


Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [33]:
#items_by_post contains all post code
items_by_post = []

for item in joined:
    if item['post_code'] not in items_by_post:
        items_by_post.append(item['post_code'])

items_by_post.sort()

groups = {pc:[] for pc in items_by_post}
for item in joined:
    groups[item['post_code']].append(item['items'])

for key in groups:
    groups[key] = sum(groups[key])
    



In [34]:
#github way
s1=sorted(groups.items(), key=lambda tup: tup[0])
print(type(s1[0]))

#my way
s2 = sorted([(k, v) for k, v in groups.items()])
print(s2[:4])

<class 'tuple'>
[('B11 4BW', 20673), ('B18 7AL', 19001), ('B21 9RY', 29103), ('B23 6DJ', 24859)]


In [ ]:
postal_totals = [('B11 4BW', 20673)] * 100

#either give s1[:100] or s2[:100] both are correct
grader.score.pw__postal_totals(s2[:100])

## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [39]:
post_code_list=[]
for script in joined:
    if script['post_code'] not in post_code_list:
        post_code_list.append(script['post_code'])
        

dict_new = {post_code:[] for post_code in post_code_list}

for script in joined:
    if len(dict_new[script['post_code']])== 2 and dict_new[script['post_code']][0] == script['bnf_name']:
        dict_new[script['post_code']][1] += script['items']
    else:
        dict_new[script['post_code']].append((script['bnf_name'],script['items']))


In [40]:
list_new = []
for key in dict_new.keys():
    for i in range(len(dict_new[key])):
        list_new.append({'post_code': key, 'bnf_name': dict_new[key][i][0], 'total':dict_new[key][i][1]})
        
total_by_item_post = {(dict_info['post_code'], dict_info['bnf_name']): dict_info['total'] for dict_info in list_new}

In [41]:
assert len(total_by_item_post) == 141196

Let's use `total_by_item_post` to find the maximum item total for each postal code. To do this, we will want to regroup `total_by_item_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_by_item_post` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in `total_by_item_post` after grouping it by `'post_code'`.

In [42]:
total_by_item_post = {}
for dict_info in list_new:
    if dict_info['post_code'] in total_by_item_post.keys():
        total_by_item_post[dict_info['post_code']] += dict_info['total']
    else:
        total_by_item_post[dict_info['post_code']] = dict_info['total']
assert len(total_by_item_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [47]:
max_item_by_post = []
for key in dict_new.keys():
    max_item_by_post.append((key, (max(dict_new[key], key=lambda x:x[1]))[0],float((max(dict_new[key], key=lambda x:x[1]))[1])/total_by_item_post[key]))

max_item_by_post = sorted(max_item_by_post, key=lambda post_code: post_code[0])

def items_by_region():
    output = max_item_by_post[:100]
    return output



[('SK11 6JL', 'Omeprazole_Cap E/C 20mg', 0.007331631401549909), ('CW5 5NX', 'Omeprazole_Cap E/C 20mg', 0.021651158594736708), ('CW1 3AW', 'Omeprazole_Cap E/C 20mg', 0.018438786971171846)]
[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.014511681903932665), ('B18 7AL', 'Amoxicillin_Cap 500mg', 0.015051839376874902), ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.01305707315397038)]


In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [ ]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [45]:
grader.score.pw__items_by_region(items_by_region())

Your score:  0.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*